<a href="https://colab.research.google.com/github/koummand/FactoryMethode/blob/master/classificationchienchat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!kaggle competitions download -c dogs-vs-cats

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [1]:
from google.colab import files
files.upload()


Saving kaggle(1).json to kaggle(1).json


{'kaggle(1).json': b'{"username":"koummandjustin","key":"369717a1d0779c39a178ebca8f102889"}'}

In [6]:
!mkdir ~/.kaggle
!cp kaggle(1).json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle(1).json
!kaggle competitions download -c dogs-vs-cats

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [5]:
!unzip -qq train.zip

unzip:  cannot find or open train.zip, train.zip.zip or train.zip.ZIP.


In [12]:
# Listing 8.6 Copie d'images dans les répertoires d'entraînement, de validation et de test
import os, shutil, pathlib
# Chemin d'accès au répertoire où l'ensemble de données d'origine a été décompressé
original_dir= pathlib.Path("train")
# Répertoire où nous allons stocker notre plus petit ensemble de données
new_base_dir= pathlib.Path("cat_vs_dog_small")

# Fonction utilitaire pour copier les images de chat (et de chien) de l'index
    # start_index à l'index end_index dans le sous­répertoire new_base_dir/
    # {subset_name}/cat (et /dog). Le "subset_name" sera soit "train",
    # "validation", ou "test".
def make_subset(subset_name, start_index, end_index):
      for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir)
        fnames = [f"{category}.{i}.jpg"
            for i in range(start_index, end_index)]
        for fname in fnames:
          shutil.copyfile(src=original_dir / fname, dst=dir / fname)

          #  Créez le sous­ensemble
      #   de formation avec le premier
      #   1 000 images de
      #   chaque catégorie.
      make_subset("train", start_index=0, end_index=1000)
      # creer les sous ensembles de
      # validation avec les 500
      # prochaines images de
      # chaque catégorie.
      make_subset("validation", start_index=1000, end_index=1500)
      # Créez le sous­ensemble de
      # test avec les 1 000 images
      # suivantes de chaque catégorie.
      make_subset("test", start_index=1500, end_index=2500)

from tensorflow import keras
from tensorflow.keras import layers

# Le modèle attend Images RVB de taille 180 × 180
inputs = keras.Input(shape=(180, 180, 3))

      # redimensionez  entrées dans la plage [0, 1] en les divisant par 255
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

      # Regardons comment les dimensions des cartes d'entités changent avec chaque successif couche:
model.summary()

       # Listing 8.8 Configuration du modèle pour l'entraînement
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

 # Listing 8.9 Utiliser image_dataset_from_directory pour lire des images
            # 1 Lisez les fichiers image.
            # 2 Décodez le contenu JPEG en grilles RVB de pixels.
            # 3 Convertissez­les en tenseurs à virgule flottante.
            # 4 Redimensionnez­les à une taille partagée (nous utiliserons 180 × 180).
            # 5 Regroupez­les par lots (nous utiliserons des lots de 32 images).
from tensorflow.keras.utils
import image_dataset_from_directory
train_dataset = image_dataset_from_directory(new_base_dir / "train", image_size=(180, 180), batch_size=32)

validation_dataset = image_dataset_from_directory( new_base_dir / "validation", image_size=(180, 180), batch_size=32)

test_dataset = image_dataset_from_directory( new_base_dir / "test", image_size=(180, 180), batch_size=32)

SyntaxError: ignored

In [ ]:
 # Nous allons considérer 1 000 échantillons, où chaque échantillon est un vecteur de taille 16 :
import numpy as np
import tensorflow as tf
random_numbers = np.random.normal(size=(1000, 16))
      # La méthode de classe from_tensor_slices() peut être
      # importer numpy en tant que
      # np importer tensorflow en tant que tf
      # utilisé pour créer un Dataset à partir d'un tableau NumPy,
      # ou un tuple ou un dict de tableaux NumPy
dataset = tf.data.Dataset.from_tensor_slices(random_numbers)
        #  Au début, notre ensemble de données ne produit que des échantillons uniques :
for i, element in enumerate(dataset):
 print(element.shape)
 if i >= 2:
  break


        #  Nous pouvons utiliser la méthode .batch() pour regrouper les données :
batched_dataset = dataset.batch(32)
for i, element in enumerate(batched_dataset):
 print(element.shape)
 if i >= 2:
  break

# Plus largement, nous avons accès à une gamme de méthodes d'ensemble de données utiles, telles que

        #  Mélange les éléments dans un tampon
# .shuffle(buffer_size)
        # pré­extrait un tampon d'éléments dans la mémoire GPU pour obtenir une meilleure utilisation de l'appareil.
# .prefetch(buffer_size)
        #  Applique une transformation arbitraire à chaque élément du ensemble de données
        #  la fonction appelable, qui s'attend à prendre en entrée un seul élément produit par l'ensemble de données).
# .map(callable)

        # La méthode .map() , en particulier, est celle que vous utiliserez souvent. Voici un exemple.
        # Nous l'utiliserons pour remodeler les éléments de notre ensemble de données de jouets de la forme (16,) à la
        # forme (4, 4) :

reshaped_dataset = dataset.map(lambda x: tf.reshape(x, (4, 4)))
for i, element in enumerate(reshaped_dataset):
  print(element.shape)
  if i >= 2:
   break



        # Listing 8.10 Affichage des formes des données et des étiquettes générées par le jeu de données
for data_batch, labels_batch in train_dataset:
  print("data batch shape:", data_batch.shape)
  print("labels batch shape:", labels_batch.shape)
  break

          # data batch shape: (32, 180, 180, 3)
          # labels batch shape: (32,)

          # ModelCheckpoint enregistrement du model apres chaque iteration
          # save_best_only=True et monitor="val_loss": enregistrement du model si et seulement si la perte de validation diminue

         # Listing 8.11 Ajustement du modèle à l'aide d'un jeu de données

callbacks = [keras.callbacks.ModelCheckpoint(
        filepath="convnet_from_scratch.keras",
        save_best_only=True,
        monitor="val_loss")]

history = model.fit(train_dataset,
            epochs=30,
            validation_data=validation_dataset,
            callbacks=callbacks)

        # Listing 8.12 Affichage des courbes de perte et de précision pendant l'entraînement

import matplotlib.pyplot as plt

accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(accuracy) + 1)

plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()

plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()

plt.show()

        # Ces parcelles sont caractéristiques du surajustement. La précision de la formation augmente de manière linéaire au fil du
        # temps, jusqu'à atteindre près de 100 %, tandis que la précision de la validation culmine à 75 %

        # Listing 8.13 Évaluation du modèle sur le jeu de test

test_model = keras.models.load_model("convnet_from_scratch.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")



        # 8.2.5 Utilisation de l'augmentation des données
        # Listing 8.14 Définir une étape d'augmentation de données à ajouter à un modèle d'image

data_augmentation = keras.Sequential([
            # applique un retournement horizontal à 50 % des images qui le traversent
  layers.RandomFlip("horizontal"),
            # fait pivoter les images d'entrée d'une valeur aléatoire dans la plage [–10 %, +10 %] (ce sont fractions d'un cercle complet ­
            # en degrés, la plage serait [–36 degrés, +36 degrés])
 layers.RandomRotation(0.1),
            # effectue un zoom avant ou arrière sur l'image selon un facteur aléatoire plage [­20%, +20%]
 layers.RandomZoom(0.2),])

        # Listing 8.15 Affichage d'images d'apprentissage augmentées de manière aléatoire
plt.figure(figsize=(10, 10))
        # Nous pouvons utiliser take(N) pour n'échantillonner que N lots de l'ensemble de données. C'est
        # revient à insérer une rupture dans la
for images, _ in train_dataset.take(1):
  for i in range(9):
            # Appliquer le augmentation scène à la lot de images.
    augmented_images = data_augmentation(images)
    ax= plt.subplot(3, 3, i + 1)
            # Affichez la première image du lot de sortie. Pour chacune des neuf itérations, il s'agit d'un
            # différentes augmentations de la même image
    plt.imshow(augmented_images[0].numpy().astype("uint8"))
    plt.axis("off")

          # Listing 8.16 Définition d'un nouveau convnet qui inclut l'augmentation et l'abandon d'image
inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(loss="binary_crossentropy",
          optimizer="rmsprop",
          metrics=["accuracy"])

          # Listing 8.17 Entraînement du convnet régularisé
callbacks = [keras.callbacks.ModelCheckpoint(
            filepath="convnet_from_scratch_with_augmentation.keras",
            save_best_only=True,
            monitor="val_loss")]

history = model.fit( train_dataset,
                  epochs=100,
                  validation_data=validation_dataset,
                  callbacks=callbacks)

        #  La précision de la validation se retrouve constamment dans la plage de 80 à 85 %, une grande


        # Listing 8.18 Évaluation du modèle sur le jeu de test
test_model = keras.models.load_model("convnet_from_scratch_with_augmentation.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")




      # Listing 8.19 Instanciation de la base convolutive VGG16
conv_base = keras.applications.vgg16.VGG16(
              # weights spécifie le point de contrôle de poids à partir duquel initialiser le modèle.
weights="imagenet",
              # include_top fait référence à l'inclusion (ou non) du classificateur densément connecté sur
              # sommet du réseau. Par défaut, ce classificateur densément connexe correspond à
              # les 1 000 cours d'ImageNet. Parce que nous avons l'intention d'utiliser notre propre densément
              # classificateur connecté (avec seulement deux classes : chat et chien), nous n'avons pas besoin de
              # l'inclure
include_top=False,
              # input_shape est la forme des tenseurs d'image que nous allons alimenter au réseau.
input_shape=(180, 180, 3))

conv_base.summary()

          # augmentation des données.
import numpy as np
def get_features_and_labels(dataset):
  all_features = []
  all_labels = []
  for images, labels in dataset:
    preprocessed_images = keras.applications.vgg16.preprocess_input(images)
    features = conv_base.predict(preprocessed_images)
    all_features.append(features)
    all_labels.append(labels)
    return np.concatenate(all_features), np.concatenate(all_labels)

train_features, train_labels = get_features_and_labels(train_dataset)

val_features, val_labels = get_features_and_labels(validation_dataset)

test_features, test_labels = get_features_and_labels(test_dataset)


train_features.shape


           #  Listing 8.21 Définition et entraînement du classifieur densément connecté
inputs = keras.Input(shape=(5, 5, 512))
                # Notez l'utilisation de la couche
                # Flatten avant de passer les
                # entités à une couche Dense.
x = layers.Flatten()(inputs)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)

model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

callbacks = [keras.callbacks.ModelCheckpoint(
            filepath="feature_extraction.keras",
            save_best_only=True,
            monitor="val_loss")]

history = model.fit(train_features, train_labels,
              epochs=20,
              validation_data=(val_features, val_labels),
              callbacks=callbacks)

                # L'apprentissage est très rapide car nous n'avons affaire qu'à deux couches Dense : une époque prend
                # moins d'une seconde même sur CPU.

                # Regardons les courbes de perte et de précision pendant l'entraînement (voir figure 8.13).

          #  Listing 8.22 Tracé des résultats
import matplotlib.pyplot as plt

acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, "bo", label="Training accuracy")
plt.plot(epochs, val_acc, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()

plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()




 #EXTRACTION DE CARACTÉRISTIQUES ET AUGMENTATION DE DONNÉES

            # Dans Keras, nous figeons(gelons) une couche ou un modèle en définissant son attribut entraînable sur False.

      # Listing 8.23 Instanciation et gel de la base convolutive VGG16
conv_base = keras.applications.vgg16.VGG16(
              weights="imagenet",
              include_top=False)

                        # La définition de trainable sur False vide la liste des pondérations entraînables de la couche ou du modèle.
conv_base.trainable = False

      # Listing 8.24 Impression de la liste des poids entraînables avant et après congélation
                         # Il s'agit du nombre de poids pouvant être entraînés avant de geler la base de conversion : 26
conv_base.trainable = True
print("Ceci est le nombre de poids entraînables "
      "avant de geler la base de conversion :",
      len(conv_base.trainable_weights))

                          # Il s'agit du nombre de poids pouvant être entraînés après avoir gelé la base de conversion : 0
conv_base.trainable = False
print("Ceci est le nombre de poids pouvant être entraînés "
      "après avoir gelé la base de conversion :",
      len(conv_base.trainable_weights))



        # Listing 8.25 Ajout d'une étape d'augmentation de données et d'un classifieur à la base convolutive
data_augmentation = keras.Sequential(
                           [layers.RandomFlip("horizontal"),
                            layers.RandomRotation(0.1),
                            layers.RandomZoom(0.2),
                          ])

inputs = keras.Input(shape=(180, 180, 3))
                        #  Appliquer l'augmentation des données.
x = data_augmentation(inputs)
x = keras.applications.vgg16.preprocess_input(x)
                        # Appliquer la mise à l'échelle de la valeur d'entrée.
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)



model.compile(loss="binary_crossentropy",
          optimizer="rmsprop",
          metrics=["accuracy"])



callbacks = [keras.callbacks.ModelCheckpoint(
      filepath="feature_extraction_with_data_augmentation.keras",
      save_best_only=True,
      monitor="val_loss")]

history = model.fit(train_dataset,
            epochs=50,
            validation_data=validation_dataset,
            callbacks=callbacks)

                #  Comme vous pouvez le constater, nous atteignons une précision
                #  de validation de plus de 98 %. C'est une nette amélioration par rapport au modèle précédent


                                      # Vérifions la précision du test.

        # Listing 8.26 Évaluation du modèle sur le jeu de test

test_model = keras.models.load_model("feature_extraction_with_data_augmentation.keras")

test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")






# 8.3.2 Affiner un modèle pré­entraîné (Une autre technique largement utilisée pour la réutilisation du modèle,)


          # Listing 8.27 Geler tous les calques jusqu'au quatrième à partir du dernier
conv_base.trainable = True
for layer in conv_base.layers[:-4]:
 layer.trainable = False



          # Listing 8.28 Affiner le modèle
model.compile(loss="binary_crossentropy",
      optimizer=keras.optimizers.RMSprop(learning_rate=1e-5),
      metrics=["accuracy"])

callbacks = [keras.callbacks.ModelCheckpoint(
      filepath="fine_tuning.keras",
      save_best_only=True,
      monitor="val_loss")]

history = model.fit(train_dataset,
              epochs=30,
              validation_data=validation_dataset,
              callbacks=callbacks)

                # On peut enfin évaluer ce modèle sur les données de test :
model = keras.models.load_model("fine_tuning.keras")

test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")


